# Исследование надежности заемщиков


## Изучение общей информации о данных

Импортируем библиотеку `pandas`. Считаем данные из csv-файла в датафрейм и сохраним в переменную `data`

In [1]:
import pandas as pd

In [2]:
try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

Выведем первые 20 строчек датафрейма `data` на экран.

In [3]:
data.head(20)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Выведем основную информацию о датафрейме с помощью метода `info()`.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

Выведем количество пропущенных значений для каждого столбца.

In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаем на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработаем значения в этом столбце: заменим все отрицательные значения положительными с помощью метода `abs()`.

In [7]:
data['days_employed'] = data['days_employed'].abs()

Для каждого типа занятости выведем медианное значение трудового стажа `days_employed` в днях.

In [8]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставим их как есть. Этот столбец не понадобится для исследования.

Выведем перечень уникальных значений столбца `children`.

In [9]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце `children` есть два аномальных значения. Удалим строки, в которых встречаются такие аномальные значения из датафрейма `data`.

In [10]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

Ещё раз выведем перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.

In [11]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5], dtype=int64)

### Удаление пропусков (продолжение)

Заполним пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.

In [12]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

Убедимся, что все пропуски заполнены.

In [13]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

Заменим вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.

In [14]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

Обработаем неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведем их к нижнему регистру. Проверим остальные столбцы.

In [15]:
data['education'] = data['education'].str.lower()

Выведем на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалим их.

In [16]:
data.duplicated().sum()

71

In [17]:
data = data.drop_duplicates()

### Категоризация данных

На основании диапазонов, указанных ниже, создадим в датафрейме `data` столбец `total_income_category` с категориями:

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [18]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [19]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

Выведем на экран перечень уникальных целей взятия кредита из столбца `purpose`.

In [20]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

Создадим функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

Используем собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучим данные в столбце `purpose` и определим, какие подстроки помогут правильно определить категорию.

In [21]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [22]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

На основании данных датафрейма определим, имеется ли зависимость между количеством детей у заемщика и возвратом кредита в срок. Для этого сгруппируем значения по количеству детей и для каждой группы вычислим отношение количества заемщиков, имевших задолженность, к общему количеству заемщиков.

In [23]:
#Создание функции для построения сгруппированного датафрейма
#в параметре функции указывается название столбца, по которому будет осуществлена группировка датафрейма
def corellation(benchmark):
    result = data.groupby(benchmark).agg({'debt':['sum', 'count']})
    #Группировка данных датафрейма по заданному критерию
    result['percent_of_debtors'] = round(((result['debt']['sum'] / result['debt']['count']) * 100), 2)
    #Вычисление отношения количества заемщиков, имевших задолженность, к общему количеству заемщиков
    return result

In [24]:
corellation('children')
#Группировка данных по количеству детей и расчет отношения количества заемщиков,
#имевших задолженность, к общему количеству заемщиков

debt        percent_of_debtors
           sum  count                   
children                                
0         1063  14091               7.54
1          444   4808               9.23
2          194   2052               9.45
3           27    330               8.18
4            4     41               9.76
5            0      9               0.00

`children` - количество детей\
`sum` - количество заемщиков, имевших задолженность\
`count` - общее количество заемщиков\
`percent_of_debtors` - доля заемщиков, имевших задолженность от их общего количества, в процентах

**Вывод:** 
в результате исследования данных установлено, что *имеется зависимость между количеством детей у заемщика и возвратом кредита в срок.*\
Доля имевших задолженность заемщиков у которых нет детей (7,54%), ниже, чем доля имевших задолженность заемщиков у которых есть 1 и более детей (8,18%-9,76%). Нулевое значение доли замемщиков с 5 детьми получено по причине недостаточного количества данных по данной категории (всего 9 позиций в предоставленных данных).

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Определим зависимость между семейным положением заемщика и возвратом кредита в срок.

In [25]:
data['family_status'] = data['family_status'].str.lower() #Приведение записей столбца 'family_status' к строчному виду 
corellation('family_status').sort_values('percent_of_debtors')
#Группировка данных с помощью ранее заданной функции по статусу семейного положения и расчет отношения количества заемщиков,
#имевших задолженность, к общему количеству заемщиков 
#Для наглядности данные отсортированы по столбцу 'percent_of_debtors'

debt        percent_of_debtors
                       sum  count                   
family_status                                       
вдовец / вдова          63    951               6.62
в разводе               84   1189               7.06
женат / замужем        927  12261               7.56
гражданский брак       385   4134               9.31
не женат / не замужем  273   2796               9.76

`family_status` - семейное положение\
`sum` - количество заемщиков, имевших задолженность\
`count` - общее количество заемщиков\
`percent_of_debtors` - доля заемщиков, имевших задолженность от их общего количества, в процентах

**Вывод:** 
в результате исследования данных установлено, что *имеется зависимость мужду семейным положением заемщика и возвратом кредита в срок.*\
Доля имевших задолженность заемщиков, имеющих статус "не женат / не замужем" (9,31%) или "гражданский брак" (9,76%) выше, чем доля имевших задолженность заемщиков, имеющих статус "вдовец / вдова" (6,62%), "в разводе" (7,06%) или "женат / замужем" (7,56%).

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Определим зависимость между уровнем дохода заемщика и возвратом кредита в срок.
Для категоризации заемщиков примем назначенные ранее диапазоны значений дохода.

In [26]:
corellation('total_income_category').sort_values('percent_of_debtors')
#Группировка данных по категории уровня дохода и расчет отношения количества заемщиков,
#имевших задолженность, к общему количеству заемщиков
#Для наглядности данные отсортированы по столбцу 'percent_of_debtors'

debt        percent_of_debtors
                        sum  count                   
total_income_category                                
D                        21    349               6.02
B                       354   5014               7.06
A                         2     25               8.00
C                      1353  15921               8.50
E                         2     22               9.09

`total_income_category` - категория дохода\
`sum` - количество заемщиков, имевших задолженность\
`count` - общее количество заемщиков\
`percent_of_debtors` - доля заемщиков, имевших задолженность от их общего количества, в процентах

Категории доходов присвоены в зависимости от суммы дохода:\
*E* — 0–30000\
*D* — 30001–50000\
*C* — 50001–200000\
*B* — 200001–1000000\
*A* — 1000001 и выше

**Вывод:**  в результате исследования данных установлено, что *имеется зависимость между уровнем дохода заемщика и возвратом кредита в срок.*\
Доля имевших задолженность заемщиков с доходом 30001-50000 (6,02%) самая низкая. Наиболее высокая доля у имевших задолженность заемщиков с доходом 50001-200000 (8,50%) и 0-30000 (9,09%). Для более точной оценки влияния данного критерия на возврат кредита в срок необходимо получить больше данных по заемщикам с уровнями дохода 0-30000, 30001-50000 и 1000001 и выше.

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Определим зависимость между уровнем дохода заемщика и возвратом кредита в срок.

In [27]:
corellation('purpose_category').sort_values('percent_of_debtors')
#Группировка данных по цели кредита и расчет отношения количества заемщиков,
#имевших задолженность, к общему количеству заемщиков
#Для наглядности данные отсортированы по столбцу 'percent_of_debtors'

debt        percent_of_debtors
                          sum  count                   
purpose_category                                       
операции с недвижимостью  780  10751               7.26
проведение свадьбы        183   2313               7.91
получение образования     369   3988               9.25
операции с автомобилем    400   4279               9.35

`purpose_category` - цель кредита\
`sum` - количество заемщиков, имевших задолженность\
`count` - общее количество заемщиков\
`percent_of_debtors` - доля заемщиков, имевших задолженность от их общего количества, в процентах

**Вывод:**  в результате исследования данных установлено, что *имеется зависимость между целью и возвратом кредита в срок.*\
Доля имевших задолженность заемщиков с целями кредита "операции с недвижимостью" (7,26%) и "проведение свадьбы" (7,91%) ниже, чем доля имевших задолженность заемщиков с целями кредита "получение образования" (9,25%) и "операции с автомобилем" (9,35%).

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* пропуски в данных в столбцах с данными "общий трудовой стаж в днях" и "ежемесячный доход" в *одних и тех же строках* присутствуют для клиентов со следующими типами занятости: пенсионер, госслужащий, компаньон, сотрудник, предприниматель. Возможно пропуски в данных обусловлены тем, что для получения кредита некоторым заемщикам из вышеперечисленных категорий необязательно указывать данные о доходах и трудовой деятельности, они заправшиваются и предоставляются другими организациями/органами.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* пропуски в количественных данных целесообразно заполнять медианным значением по группе или медианным по всем значениям (в зависимости от полноты и структуры данных), поскольку заполнение их средними значениями может исказить результаты исследования (значения с аномальными отклонениями в данных вызвают значительное увеличение или уменьшение среднего показателя). Заполнение пропусков нулевыми, максимальными или минимальными значениями по группе или по всем данным также приведет к искажению результатов исследования.

### Общий вывод.

На основании предоставленных банком данных - статистики о платежеспособности клиентов выполнено исследование, целью которого являлось выявление факторов, влияющих на возврат кредита заемщиком в срок. Данные исследования необходимы для построения модели кредитного скоринга - специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Для выявления факторов, влияющих на возврат кредита заемщиком в срок, в ходе исследования были проверены гипотезы о наличии зависимости между: количеством детей и возвратом кредита в срок; семейным положением и возвратом кредита в срок; между уровнем дохода и возвратом кредита в срок, между целью кредита и возвратом его в срок.


**Возврат кредита в срок зависит от наличия или отсутствия детей у заемщика.**

Доля просроченных кредитов у заемщиков с детьми выше, чем у заемщиков, не имеющих детей.

Доля имевших задолженность заемщиков у которых нет детей составляет 7,54%, в то время как доля имевших задолженность заемщиков у которых есть 1 и более детей составляет от 8,18% до 9,76%.

*Для установления более точной зависимости рекомендуется получить больше данных по заемщикам с 3, 4 и 5 детьми.*

**Возврат кредита в срок зависит от семейного положения заемщика.**

Доля просроченных кредитов у заемщиков не состовших/не состоящих в официальном браке выше, чем у состоявших/состоящих в официальном браке.

Заемщики, не состоявшие/не состоявшие в официальном браке в зависимости от статуса семейного положения имеют следующие доли просроченных кредитов: "не женат/не замужем" - 9,31%, "гражданский брак" - 9,76%. В то же время заемщики, состоящие/состоявшие в официальном браке в зависимости от статуса имеют следующие доли: "довец/вдова" - 6,62%, "в разводе" - 7,06%, "женат/замужем" - 7,56%.

**Возврат кредита в срок незначительно зависит от уровня дохода заемщика.**

На основании предоставленных данных выявлено следующее:\
доля имевших задолженность заемщиков с доходом 30001-50000 - 6,02% самая низкая;\
доля имевших задолженность заемщиков с доходом 200001-1000000 - 7,06%;\
доля имевших задолженность заемщиков с доходом 1000001 и выше - 8,00%;\
наиболее высокая доля у имевших задолженность заемщиков с доходом 50001-200000 - 8,50% и 0-30000 - 9,09%;\
Однозначно сказать, что уровень дохода заемщика влияет на возврат им кредита в срок невозможно.

*Для установления более точной зависимости рекомендуется получить больше данных по заемщикам с уровнями дохода 0-30000, 30001-50000 и 1000001 и выше.*
*До получения вышеуказанных данных и проведения повторного исследования критерий уровня дохода в модели кредитного скоринга применять не рекомендуется.*

**Возврат кредита в срок зависит от цели кредита.**

Доля имевших задолженность заемщиков с целями кредита "операции с недвижимостью" - 7,26% и "проведение свадьбы" - 7,91% ниже, чем доля имевших задолженность заемщиков с целями кредита "получение образования" - 9,25% и "операции с автомобилем" - 9,35%.
\
\
\
***Для построения модели кредитного скоринга рекомендуется использовать такие критерии оценки потенциального заемщика, как количество детей, семейное положение и цель кредита.***